# HPC/QC Architectures: Introduction To Qiskit

|   |   |
|---|---|
| Author  | Christoph Schober  |
|  Last Update | 2023-12-12  |

Goals:
* Basic introduction to Qiskit
* Focus on lecture content, no universal tutorial

## Suggested Literature


### Programming Quantum Computers
* Eric R. Johnston, Nic Harrigan, Mercedes Gimeno-Segovia.
* [LINK](https://www.oreilly.com/library/view/programming-quantum-computers/9781492039679/)
* Focus on programming and algorithm
* Almost no physics or math
* Uses visual 'circle representation' to understand algorithms step by step
* Online Quantum Simulator: https://oreilly-qc.github.io/#
  
![Programming](https://learning.oreilly.com/library/cover/9781492039679/250w/)

### Dancing with Qubits: 
* Robert S. Sutor.
* [LINK](https://www.packtpub.com/product/dancing-with-qubits/9781838827366)
* Comprehensive introduciton to Quantum Computing including the basic math and some physics
* Few algorithms explained in more detail

![Dancing](https://content.packt.com/B14705/cover_image_small.png)  

## Quantum Circuits
Qiskit's main functionality to create Quantum Circuits is the [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) class. You can use it to create circuits with qubits and classical bits

In [ ]:
from qiskit import QuantumCircuit
# Create a circuit with 2 qubits
circuit_2 = QuantumCircuit(2)

# Create a circuit with 2 qubits and 3 classical bits
circuit_2_3 = QuantumCircuit(2, 3)

A very important feature is the ability to visualize any circuit using `QuantumCircuit.draw()`. In our case the circuit is empty because we did not yet add any gates.

In [ ]:
circuit_2_3.draw()

## Gates
Qiskit has all of the basic gates and many 'exotic' gates available to be used. For basic gates shortcuts are available directly on the QuantumCircuit

In [ ]:
circuit_2_3 = QuantumCircuit(2, 3)

# X on the first qubit
circuit_2_3.x(0)
# CNOT between first and second qubit
circuit_2_3.cx(0, 1)

# Draw (using the nicer matplotlib-output style)
circuit_2_3.draw(output="mpl", style="iqp")

Some more special gates must be 'appended' to a circuit using an explicit class instance:

In [ ]:
from qiskit.circuit.library import C3XGate
spec = QuantumCircuit(4)
spec.append(C3XGate(), [0,1,2,3])
spec.draw(output="mpl", style="iqp")

A list of available gates and more complex building blocks can be found here: https://docs.quantum.ibm.com/api/qiskit/circuit_library#standard-gates

## Measurements
Depending on the requirements it is possible to measure individual qubits (also mid-circuit) or measure all qubits at the end of the circuit.

In [ ]:
circuit_2_3 = QuantumCircuit(2, 3)

# X on the first qubit
circuit_2_3.x(0)
# CNOT between first and second qubit
circuit_2_3.cx(0, 1)

# measure qbit 0 and store value in classical bit 0
circuit_2_3.measure(0,0)
circuit_2_3.measure(1,2)
circuit_2_3.draw(output="mpl", style="iqp")

In [ ]:
circuit_2_3 = QuantumCircuit(2, 3)

# X on the first qubit
circuit_2_3.x(0)
# CNOT between first and second qubit
circuit_2_3.cx(0, 1)
# measure qbit 1 and store value in classical bit 2
circuit_2_3.measure(1,2)
circuit_2_3.draw(output="mpl", style="iqp")

Using `.measure_all()` will introduce a barrier and necessary classical bits to store the measured values automatically

In [ ]:
circuit_2_3 = QuantumCircuit(2)

# X on the first qubit
circuit_2_3.x(0)
# CNOT between first and second qubit
circuit_2_3.cx(0, 1)

circuit_2_3.measure_all()
circuit_2_3.draw()

<div class="alert alert-block alert-info">
<h3>Exercise 1</h3>
    <p>Use the skeleton code in the next box and create the following circuit:</p>
    <img src="images/exercise1_circuit.png" width="30%">
</div>

In [ ]:
### BEGIN SOLUTION ###
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.h(1)
qc.t(1)
qc.h(1)
qc.cx(0, 1)

### END SOLUTION

# Measurements with barriers to structure the circuit
qc.barrier()
qc.measure(0, 0)
qc.barrier()
qc.measure(0, 0)
qc.draw(output="mpl", style="iqp")

In [ ]:
# Example barrier use
import numpy as np

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.h(1)
qc.x(1)
qc.t(1)
qc.rz(np.pi/2, 1)

qc.barrier()
qc.measure(0,0)
qc.measure(1,1)
qc.draw(output="mpl", style="iqp")

In [ ]:
# Example barrier use
import numpy as np

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.h(1)
qc.x(1)
qc.t(1)
qc.rz(np.pi/2, 1)

qc.measure(0,0)
qc.measure(1,1)
qc.draw(output="mpl", style="iqp")

<div class="alert alert-block alert-info">End of Exercise</div>

## Executing the circuit
A circuit in Qiskit can be run by different backends, such as noisy or ideal simulators or even real quantum hardware.

For any teaching or learning it is usually most efficient to use the simulator backends.

In [ ]:
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram, plot_state_city, plot_state_qsphere, plot_bloch_vector
from qiskit.visualization import array_to_latex

# Create a simulator object to be used
simulator = Aer.get_backend('aer_simulator')

# Create a simple circuit with entangled qubits
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0,1)
bell.measure_all()
bell.draw(output="mpl", style="iqp")

The circuit is run using the defined simulator and results can be fetched immediately (for real hardware there can be a delay of minutes to days depending on the queue size).

**Important: Since quantum circuits are probabilistic multiple runs or "shots" are required to get a statistically significant result!**

In [ ]:
result = simulator.run(bell).result()
result.get_counts()

The default amount of runs in Qiskit is usually **1024**. This can be configured via the `shots` argument to the `.run`-call:

In [ ]:
result2 = simulator.run(bell, shots=1).result()
result2.get_counts()

The result can also be visualized using `plot_histogram`:

In [ ]:
plot_histogram(result.get_counts())

<div class="alert alert-block alert-info">
<h3>Exercise 2</h3>
    <p>Create an entangled circuit that will return <i>00</i> more often than <i>11</i> ("has a different probability for measuring <i>00</i> than <i>11</i>") and test the circuit by measuring and plotting the result using the aer_simulator</p>
    <ul>
        <li>Which gates influence the probability of measuring a certain value?</li>
        <li>Hint: Rotations in Qiskit are defined in Radians. You can use fractions of $\pi$ via <code>np.pi / 4</code></li>
    </ul>
</div>

In [ ]:
import numpy as np
simulator = Aer.get_backend('aer_simulator')

## BEGIN SOLUTION ##
# Create a simple circuit with entangled qubits
bell = QuantumCircuit(2)
bell.h(0)
bell.ry(np.pi/6, 0)
bell.cx(0,1)

bell.measure_all()
bell.draw()

## END SOLUTION ## 

In [ ]:
## BEGIN SOLUTION ##
result = simulator.run(bell, shots=1024).result()
counts = result.get_counts()
print(counts)
print(counts["11"]/counts["00"] * 100)
plot_histogram(result.get_counts())
## END SOLUTION ## 

<div class="alert alert-block alert-info">End of Exercise</div>

### Debugging Insights Using Simulators
One benefit of the simulators is the option to get insights into the state of the computation within the circuit by storing the so-called statevector (or wavefunction) of the circuit at a specific point. This is useful to gain more insights into what is happening.

Note: This is of course **only** possible on simulators!

In [ ]:
from qiskit.quantum_info import Statevector

# Create a simple circuit with entangled qubits
bell = QuantumCircuit(2)
stv0 = Statevector.from_instruction(bell)
bell.h(0)
stv1 = Statevector.from_instruction(bell)
bell.cx(0,1)
stv2 = Statevector.from_instruction(bell)
bell.measure_all()
bell.draw(output="mpl", style="iqp")

The statevector can be printed directly

In [ ]:
print(stv0)

or in a nicer format using LaTeX:

In [ ]:
from qiskit.visualization import array_to_latex
# Initial state with qubits initialized to 0
array_to_latex(stv0)

In [ ]:
# After Hadamard on q0
array_to_latex(stv1)

In [ ]:
# after CNOT between q0 and q1 (bell state)
array_to_latex(stv2)

The statevector can also be visualized using different plotting features such as the so-called "city map":

In [ ]:
plot_state_city(stv0)

In [ ]:
plot_state_city(stv1)

In [ ]:
plot_state_city(stv2)

There are different visualization options available: https://docs.quantum.ibm.com/build/circuit-visualization#plot-state-

## Running with noise

In [ ]:
# Construct the noise model from backend properties
from qiskit import execute
from qiskit.providers.fake_provider import FakeVigoV2
from qiskit.providers.aer.noise import NoiseModel

device_backend = FakeVigoV2()

# The device coupling map is needed for transpiling to correct
# CNOT gates before simulation
coupling_map = device_backend.coupling_map

noise_model = NoiseModel.from_backend(device_backend)
print(noise_model)

# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('aer_simulator')

result_noise = execute(bell, simulator,
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates).result()

In [ ]:
plot_histogram(result_noise.get_counts())

Here we can clearly see the result of the noise: The states `01` and `10` are not possible for a pure Bell-state circuit.

## Qiskit Qubit Ordering
Just as for classic hardware there is also the convention of "Endianness" (see https://en.wikipedia.org/wiki/Endianness for general background) in Quantum Computing. Qiskit uses the "little endian" convention, this means, the least significant bit is on the right.

Circuit plots, on the other hand, are visualized "as expected". This can lead to some confusion.

Let's see an example:

In [ ]:
circ = QuantumCircuit(3)
circ.x(0)  # x on qubit 0
circ.barrier()
circ.x(1)  # x on qubit 1
circ.barrier()
circ.x(2)  # x on qubit 2

circ.measure_all()
circ.draw(output="mpl", style="iqp")

In [ ]:
# Define helper function to measure circuits with different qubits being X'ed
def do_x_on(qubit_idx):
    circ = QuantumCircuit(3)
    circ.x(qubit_idx)
    circ.measure_all()
    result = simulator.run(circ).result()
    print(f"Result for X on Qubit with index {qubit_idx}: {list(result.get_counts().keys())[0]}")
    return circ

In [ ]:
circ = do_x_on(0)
circ.draw(output="mpl", style="iqp")

Carefully compare the string `Result for X on Qubit with index 0: 001` with the plot! `q0` is the last bit in the string, **not** the first!

In [ ]:
do_x_on(0)
do_x_on(1)
do_x_on(2)

### Effect on matrix representations
The ordering also influences how the matrix representations of gates and any unitaries in Qiskit are constructed. They will look different from the text-book representation that you are used to.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

circ = QuantumCircuit(2)
circ.cx(0, 1)
circ.draw(output="mpl", style="iqp")

In [ ]:
print('Little endian (QISKIT):')
array_to_latex(Operator(circ), prefix="CNOT=")


In [ ]:
print('Big endian (TEXTBOOKS):')
array_to_latex(Operator(circ.reverse_bits()), prefix="CNOT=")

<div class="alert alert-block alert-warning">
<h3>Keep this in mind when working with Qiskit and interacting with qubit-strings or gate representations!</h3>
</div>